In [1]:
import pandas as pd
import numpy as np
import os
import glob
import json
import time
from collections import defaultdict
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb

## Configuration

In [16]:
# Number of force cells in the robotic leg
N_CELLS = 8

# Path where the results are stored
RESULTS_PATH = '../../../../results'
# ID of the training and validation data resulting from this notebook, stored in RESULTS_PATH
DATA_ID = '0005_19042021'
# Hyperparameters search date
HS_DATE = '20042021'
# Number of folds in cross-validation
CV = 6

print('Model trained with data: ' + DATA_ID)

pd.set_option('display.max_columns', None)

Model trained with data: 0005_19042021


## Hyperparameters seach analysis

In [17]:
results_files_ls = glob.glob(os.path.join(RESULTS_PATH, DATA_ID, 'XGB_{}'.format(HS_DATE), 'XGB_{}_*.json'.format(HS_DATE)))

print('Number of results files: {}'.format(len(results_files_ls)))

Number of results files: 123


In [18]:
# Load all the results and generates a pandas dataframe
results_ls = []
for results_file in results_files_ls:
    with open(results_file) as json_file:
        results_dict = json.load(json_file)
        
    dict_aux = {}
    dict_aux['params_ID'] = results_dict['id']
    for key, value in results_dict['parameters'].items():
        dict_aux['param_' + key] = value
    for key, value in results_dict['cv_results'].items():
        dict_aux['__'.join([key, 'mean'])] = np.mean(value)
        dict_aux['__'.join([key, 'std'])] = np.std(value)

    results_ls.append(dict_aux)
        
results_df = pd.DataFrame(results_ls)
results_df

,params_ID,param_booster,param_eta,param_gamma,param_lambda,param_max_depth,param_nthread,param_objective,param_seed,param_subsample,fit_time__mean,fit_time__std,Train_Fx_MAE_mean__mean,Train_Fx_MAE_mean__std,Train_Fx_MAE_std__mean,Train_Fx_MAE_std__std,Train_Fx_MSE_mean__mean,Train_Fx_MSE_mean__std,Train_Fx_MSE_std__mean,Train_Fx_MSE_std__std,Train_Fx_R2_mean__mean,Train_Fx_R2_mean__std,Train_Fx_R2_std__mean,Train_Fx_R2_std__std,Train_Fy_MAE_mean__mean,Train_Fy_MAE_mean__std,Train_Fy_MAE_std__mean,Train_Fy_MAE_std__std,Train_Fy_MSE_mean__mean,Train_Fy_MSE_mean__std,Train_Fy_MSE_std__mean,Train_Fy_MSE_std__std,Train_Fy_R2_mean__mean,Train_Fy_R2_mean__std,Train_Fy_R2_std__mean,Train_Fy_R2_std__std,Train_Fz_MAE_mean__mean,Train_Fz_MAE_mean__std,Train_Fz_MAE_std__mean,Train_Fz_MAE_std__std,Train_Fz_MSE_mean__mean,Train_Fz_MSE_mean__std,Train_Fz_MSE_std__mean,Train_Fz_MSE_std__std,Train_Fz_R2_mean__mean,Train_Fz_R2_mean__std,Train_Fz_R2_std__mean,Train_Fz_R2_std__std,Valid_Fx_MAE_mean__mean,Valid_Fx_MAE_mean__std,Valid_Fx_MAE_std__mean,Valid_Fx_MAE_std__std,Valid_Fx_MSE_mean__mean,Valid_Fx_MSE_mean__std,Valid_Fx_MSE_std__mean,Valid_Fx_MSE_std__std,Valid_Fx_R2_mean__mean,Valid_Fx_R2_mean__std,Valid_Fx_R2_std__mean,Valid_Fx_R2_std__std,Valid_Fy_MAE_mean__mean,Valid_Fy_MAE_mean__std,Valid_Fy_MAE_std__mean,Valid_Fy_MAE_std__std,Valid_Fy_MSE_mean__mean,Valid_Fy_MSE_mean__std,Valid_Fy_MSE_std__mean,Valid_Fy_MSE_std__std,Valid_Fy_R2_mean__mean,Valid_Fy_R2_mean__std,Valid_Fy_R2_std__mean,Valid_Fy_R2_std__std,Valid_Fz_MAE_mean__mean,Valid_Fz_MAE_mean__std,Valid_Fz_MAE_std__mean,Valid_Fz_MAE_std__std,Valid_Fz_MSE_mean__mean,Valid_Fz_MSE_mean__std,Valid_Fz_MSE_std__mean,Valid_Fz_MSE_std__std,Valid_Fz_R2_mean__mean,Valid_Fz_R2_mean__std,Valid_Fz_R2_std__mean,Valid_Fz_R2_std__std
0,69JLNJO5RI,gbtree,0.5,0.010,1,3,8,reg:squarederror,0,0.75,65.098543,1.218346,8.660381,0.308246,2.542358,0.517207,163.166964,15.163582,100.421780,28.924884,0.455936,0.027535,0.109046,0.025455,7.776703,0.415907,4.719952,0.558537,169.018689,27.446492,181.757054,44.655947,0.343985,0.033728,0.125440,0.013805,13.421215,0.411529,5.079392,0.202952,391.478448,16.918628,331.995217,16.365414,0.471004,0.034518,0.093580,0.047410,9.668765,0.416967,2.846736,0.268242,198.910371,18.467043,118.662945,27.952420,0.300448,0.036133,0.124305,0.040222,8.220168,0.223061,4.893021,0.350033,190.293417,13.029829,207.056361,16.405125,0.212439,0.047328,0.154688,0.018848,15.182772,0.608899,6.123708,0.600148,513.558613,35.236396,485.601923,86.764550,0.289525,0.049084,0.157594,0.087675
1,FTN2A7XGI1,gbtree,0.3,0.010,2,4,8,reg:squarederror,0,0.50,72.803450,1.055087,8.131550,0.170416,2.320002,0.143161,140.605485,5.791586,79.040401,7.089656,0.523109,0.016271,0.079205,0.005475,7.394806,0.301609,4.368400,0.435084,148.119109,17.663247,153.588425,30.499090,0.399236,0.019840,0.121197,0.018887,12.614450,0.425561,4.647628,0.166900,345.375069,19.869002,278.672046,15.135022,0.525071,0.035966,0.082801,0.036251,9.504354,0.464097,2.783744,0.366424,193.427923,19.979574,113.535239,28.901364,0.319120,0.040427,0.116487,0.029223,8.097245,0.225779,4.728623,0.344174,184.085513,14.206924,197.393287,18.571446,0.220997,0.044704,0.162695,0.019789,14.870117,0.733362,5.960564,0.685565,495.101110,32.770727,458.727406,78.059753,0.309798,0.052237,0.163587,0.092558
2,YO1GY3J97Z,gbtree,0.3,0.005,2,6,8,reg:squarederror,0,0.75,131.329646,2.483775,6.801254,0.268628,2.042166,0.329952,98.394610,8.178699,58.186013,13.196209,0.663714,0.022251,0.080217,0.012457,6.538690,0.393707,3.821124,0.475121,114.196796,17.691105,116.904412,29.840202,0.521767,0.032929,0.126657,0.023254,10.236166,0.465187,3.646358,0.284529,229.639473,19.016817,166.033110,11.773164,0.670938,0.030992,0.074857,0.034168,9.082600,0.507941,2.639738,0.316877,180.933245,23.465321,103.559918,27.227847,0.356618,0.048333,0.131221,0.023144,7.909461,0.230306,4.543698,0.441839,178.323619,15.424788,187.722294,23.132875,0.227755,0.056994,0.180358,0.025686,14.282780,0.639077,5.739485,0.7

In [19]:
# Sum up the scores by force axis in only one sortable score
for subset in ['Train', 'Valid']:
    for loss in ['MAE', 'MSE', 'R2']:
        results_df[subset + '_' + loss] = results_df[[subset + '_' + force + '_' + loss + '_mean__mean' for force in ['Fx', 'Fy', 'Fz']]].mean(axis=1)

In [20]:
# Sort the dataframe by the most relevant score
results_df = results_df.sort_values(['Valid_R2'], ascending=False)
results_df

,params_ID,param_booster,param_eta,param_gamma,param_lambda,param_max_depth,param_nthread,param_objective,param_seed,param_subsample,fit_time__mean,fit_time__std,Train_Fx_MAE_mean__mean,Train_Fx_MAE_mean__std,Train_Fx_MAE_std__mean,Train_Fx_MAE_std__std,Train_Fx_MSE_mean__mean,Train_Fx_MSE_mean__std,Train_Fx_MSE_std__mean,Train_Fx_MSE_std__std,Train_Fx_R2_mean__mean,Train_Fx_R2_mean__std,Train_Fx_R2_std__mean,Train_Fx_R2_std__std,Train_Fy_MAE_mean__mean,Train_Fy_MAE_mean__std,Train_Fy_MAE_std__mean,Train_Fy_MAE_std__std,Train_Fy_MSE_mean__mean,Train_Fy_MSE_mean__std,Train_Fy_MSE_std__mean,Train_Fy_MSE_std__std,Train_Fy_R2_mean__mean,Train_Fy_R2_mean__std,Train_Fy_R2_std__mean,Train_Fy_R2_std__std,Train_Fz_MAE_mean__mean,Train_Fz_MAE_mean__std,Train_Fz_MAE_std__mean,Train_Fz_MAE_std__std,Train_Fz_MSE_mean__mean,Train_Fz_MSE_mean__std,Train_Fz_MSE_std__mean,Train_Fz_MSE_std__std,Train_Fz_R2_mean__mean,Train_Fz_R2_mean__std,Train_Fz_R2_std__mean,Train_Fz_R2_std__std,Valid_Fx_MAE_mean__mean,Valid_Fx_MAE_mean__std,Valid_Fx_MAE_std__mean,Valid_Fx_MAE_std__std,Valid_Fx_MSE_mean__mean,Valid_Fx_MSE_mean__std,Valid_Fx_MSE_std__mean,Valid_Fx_MSE_std__std,Valid_Fx_R2_mean__mean,Valid_Fx_R2_mean__std,Valid_Fx_R2_std__mean,Valid_Fx_R2_std__std,Valid_Fy_MAE_mean__mean,Valid_Fy_MAE_mean__std,Valid_Fy_MAE_std__mean,Valid_Fy_MAE_std__std,Valid_Fy_MSE_mean__mean,Valid_Fy_MSE_mean__std,Valid_Fy_MSE_std__mean,Valid_Fy_MSE_std__std,Valid_Fy_R2_mean__mean,Valid_Fy_R2_mean__std,Valid_Fy_R2_std__mean,Valid_Fy_R2_std__std,Valid_Fz_MAE_mean__mean,Valid_Fz_MAE_mean__std,Valid_Fz_MAE_std__mean,Valid_Fz_MAE_std__std,Valid_Fz_MSE_mean__mean,Valid_Fz_MSE_mean__std,Valid_Fz_MSE_std__mean,Valid_Fz_MSE_std__std,Valid_Fz_R2_mean__mean,Valid_Fz_R2_mean__std,Valid_Fz_R2_std__mean,Valid_Fz_R2_std__std,Train_MAE,Train_MSE,Train_R2,Valid_MAE,Valid_MSE,Valid_R2
4,UIIA5POF5I,gbtree,0.3,0.010,2,7,8,reg:squarederror,0,0.75,155.088028,3.172026,6.294416,0.240647,2.021956,0.324676,86.543521,6.616950,54.718179,11.961837,0.707899,0.019719,0.086159,0.010028,6.097978,0.485892,3.572792,0.554560,97.794590,19.388475,99.352360,31.542105,0.577916,0.044321,0.138918,0.040608,9.216268,0.554022,3.218922,0.285926,186.932261,20.638034,125.747019,14.075482,0.723520,0.036936,0.076536,0.032008,8.978160,0.467510,2.542490,0.331031,178.388202,23.855861,99.675800,27.393087,0.359898,0.053070,0.141001,0.025642,7.842939,0.219761,4.432838,0.368441,176.224368,15.657473,184.430153,23.924132,0.225376,0.055442,0.186949,0.028756,14.157323,0.660834,5.677264,0.756870,474.755313,30.919891,438.566749,65.050225,0.337478,0.050418,0.181055,0.097151,7.202888,123.756791,0.669778,10.326141,276.455961,0.307584
121,ZDMKGSRAOE,gbtree,0.3,0.005,2,7,8,reg:squarederror,0,0.75,153.540638,3.200350,6.294416,0.240647,2.021956,0.324676,86.543521,6.616950,54.718179,11.961837,0.707899,0.019719,0.086159,0.010028,6.097978,0.485892,3.572792,0.554560,97.794590,19.388475,99.352360,31.542105,0.577916,0.044321,0.138918,0.040608,9.216268,0.554022,3.218922,0.285926,186.932261,20.638034,125.747019,14.075482,0.723520,0.036936,0.076536,0.032008,8.978160,0.467510,2.542490,0.331031,178.388202,23.855861,99.675800,27.393087,0.359898,0.053070,0.141001,0.025642,7.842939,0.219761,4.432838,0.368441,176.224368,15.657473,184.430153,23.924132,0.225376,0.055442,0.186949,0.028756,14.157323,0.660834,5.677264,0.756870,474.755313,30.919891,438.566749,65.050225,0.337478,0.050418,0.181055,0.097151,7.202888,123.756791,0.669778,10.326141,276.455961,0.307584
75,TKZ3SX54J2,gbtree,0.3,0.050,2,7,8,reg:squarederror,0,0.75,153.731990,3.019236,6.294416,0.240647,2.021956,0.324676,86.543521,6.616950,54.718179,11.961837,0.707899,0.019719,0.086159,0.010028,6.097978,0.485892,3.572792,0.554560,97.794590,19.388475,99.352360,31.542105,0.577916,0.044321,0.138918,0.040608,9.216268,0.554022,3.218922,0.285926,186.932261,20.638034,125.747019,14.075482,0.723520,0.036936,0.076536,0.032008,8.978160,0.467510,2.542490,0.331031,178.388202,23.855861,99.675800,27.393087,0.359898,0.0530

In [21]:
best_params = dict(results_df.iloc[0][[col for col in results_df.columns if 'param_' in col]])
best_params = {key.replace('param_', ''): value for key, value in best_params.items()}
print('Best parameters: {}'.format(best_params))

Best parameters: {'booster': 'gbtree', 'eta': 0.3, 'gamma': 0.01, 'lambda': 2, 'max_depth': 7, 'nthread': 8, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.75}


## Best model

In [22]:
# Load data
X_train = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'X_train_{}.npy'.format(DATA_ID)))
X_test = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'X_test_{}.npy'.format(DATA_ID)))
Y_train = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'Y_train_{}.npy'.format(DATA_ID)))
Y_test = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'Y_test_{}.npy'.format(DATA_ID)))

In [25]:
results = defaultdict(list)
tr_time = []
for target in range(Y_train.shape[1]):

    dtrain = xgb.DMatrix(data=X_train, label=Y_train[:, target])
    dtest = xgb.DMatrix(data=X_test, label=Y_test[:, target])

    callbacks = []#[xgb.callback.EarlyStopping(rounds=5, metric_name='rmse', maximize=False, save_best=True)]
    
    t_start = time.time()
    model = xgb.train(best_params, dtrain, evals=[(dtest, 'rmse')], callbacks=callbacks, verbose_eval=False)
    tr_time.append(time.time() - t_start)
    
    # Save the model
    model.save_model(os.path.join(RESULTS_PATH, DATA_ID, 'XGB_{}'.format(HS_DATE), 'XGB_best_model_{}_{}_{}.joblib'.format(target, HS_DATE, DATA_ID)))
    
    train_preds = model.predict(dtrain)
    test_preds = model.predict(dtest)

    results['Train_MAE'].append(mean_absolute_error(Y_train[:, target], train_preds))
    results['Train_MSE'].append(mean_squared_error(Y_train[:, target], train_preds))
    results['Train_R2'].append(r2_score(Y_train[:, target], train_preds))
    results['Test_MAE'].append(mean_absolute_error(Y_test[:, target], test_preds))
    results['Test_MSE'].append(mean_squared_error(Y_test[:, target], test_preds))
    results['Test_R2'].append(r2_score(Y_test[:, target], test_preds))

print('Training time: {:.4f}'.format(sum(tr_time)))

Training time: 421.2417


In [26]:
# # Display the score for each axis of each force cell
# for subset in ['Train', 'Test']:
#     for f, force in enumerate(['Fx', 'Fy', 'Fz']):
#         for c in range(N_CELLS):
#             for loss in ['MAE', 'MSE', 'R2']:
#                 scores = [results[subset][loss][i + f] for i in range(0, N_CELLS * 3, 3)]
#                 print('{} {}{}{} {}: {:.4f}'.format(subset, force[0], c + 1, force[-1], loss, scores[c]))
# print('\n')

# Display the score mean and standard deviation of each axis
for subset in ['Train', 'Test']:
    for f, force in enumerate(['Fx', 'Fy', 'Fz']):
        for loss in ['MAE', 'MSE', 'R2']:
            scores = [results['_'.join([subset, loss])][i + f] for i in range(0, N_CELLS * 3, 3)]
            print(' '.join([subset, force, loss]) + ': {:.4f} ± {:.4f}'.format(np.mean(scores), np.std(scores)))

Train Fx MAE: 5.9629 ± 1.6847
Train Fx MSE: 76.4576 ± 42.7314
Train Fx R2: 0.7376 ± 0.0517
Train Fy MAE: 5.7177 ± 3.1862
Train Fy MSE: 84.2443 ± 78.6247
Train Fy R2: 0.6201 ± 0.1068
Train Fz MAE: 9.1344 ± 3.2695
Train Fz MSE: 185.7676 ± 132.3666
Train Fz R2: 0.7378 ± 0.0519
Test Fx MAE: 11.0853 ± 3.8290
Test Fx MSE: 273.7327 ± 182.5694
Test Fx R2: 0.1542 ± 0.2007
Test Fy MAE: 9.8996 ± 6.3373
Test Fy MSE: 324.9241 ± 389.9810
Test Fy R2: 0.0324 ± 0.1978
Test Fz MAE: 16.9357 ± 7.2828
Test Fz MSE: 685.7255 ± 688.0254
Test Fz R2: 0.1111 ± 0.2064


In [ ]:
# plt.figure(figsize=(20,15))
# plt.scatter(Y_train[:, 3], Y_train[:, 4], label='true', alpha=0.3)
# plt.scatter(train_preds[:, 3], train_preds[:, 4], label='preds', alpha=0.3)
# plt.legend()
# plt.show()

# plt.figure(figsize=(20,15))
# plt.scatter(Y_train[:100, 3], Y_train[:100, 4], label='true', alpha=0.3)
# plt.scatter(train_preds[:100, 3], train_preds[:100, 4], label='preds', alpha=0.3)
# plt.legend()
# plt.show()

# plt.figure(figsize=(20,15))
# plt.scatter(Y_test[:, 3], Y_test[:, 4], label='true', alpha=0.3)
# plt.scatter(test_preds[:, 3], test_preds[:, 4], label='preds', alpha=0.3)
# plt.legend()
# plt.show()